In [1]:
from casadi import *
import numpy as np

In [39]:
omega_max = 3000
tau_max = 0.05
t_total = 0.2


opti = Opti()

X = opti.variable(11) # decision variables
r_tsa = X[0]
phi_0 = X[1]
L_tsa0 = X[2]
F_spr0 = X[3]
k = X[4]
r1 = X[5]
r2 = X[6]
r3 = X[7]
theta3 = X[8]
x_tsa = X[9]
r12 = X[10]


phi = MX.sym('phi')

# 100 operations
x0 = r_tsa**2
x1 = r1**2
x2 = x_tsa**2
x3 = -L_tsa0**2 + x1 + x2
x4 = (1/2)/r1
x5 = x4/x_tsa
x6 = r2**2
x7 = theta3 + acos(x4*(-r12**2 + x1 + x6)/r2)
x8 = r2*r3
x9 = 2*x8
x10 = r3**2 + x6
x11 = -sqrt(x10 - x9*cos(x7 - acos(x5*(-phi_0**2*x0 - x3))))
x12 = -x0*(phi + phi_0)**2 - x3
x13 = x7 - acos(x12*x5)
x14 = sqrt(x10 - x9*cos(x13))
E_f = (1/2)*(F_spr0 + k*(x11 + sqrt(x10 - x9*cos(x7 - acos(x5*(-x0*((1/2)*omega_max*t_total + phi_0)**2 - x3))))))**2/k
tau_a = -x0*x5*x8*(F_spr0 + k*(x11 + x14))*(2*phi + 2*phi_0)*sin(x13)/(x14*sqrt(1 - 1/4*x12**2/(x1*x2)))

F_tau_a = Function("F_tau_a", [phi, X], [tau_a])


opti.minimize(-E_f)

opti.set_initial(X, [0.000128, 20, 0.0516, 9.46, 4.6, 0.0263, 0.015, 0.01906, -1.327, 0.0256, 0.0244])

for phi_test in np.linspace(1, 2, 10):
    opti.subject_to(opti.bounded(0, F_tau_a(phi_test, X), tau_max))

opti.subject_to(x10 - x9*cos(x7 - acos(x5*(-phi_0**2*x0 - x3))) > 0)
opti.subject_to(x10 - x9*cos(x7 - acos(x5*(-x0*((1/2)*omega_max*t_total + phi_0)**2 - x3))) > 0)
# opti.subject_to(x10 - x9*cos(x13) > 0)

log = []
opti.callback(lambda i: log.append(opti.debug.value(X))) #for logging each iteration


opts = {'ipopt.print_level':0, 'print_time':0}
opti.solver('ipopt', opts)
sol = opti.solve()

CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 11, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 11, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 11, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2023-10-27 21:15:25 WARNING("solver:nlp_g failed: NaN detected for output g, at (row 11, col 0

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:157:
.../casadi/core/optistack_internal.cpp:1000: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'

In [37]:
opti.debug.value(cos(x7 - acos(x5*(-x0*((1/2)*omega_max*t_total + phi_0)**2 - x3))))

nan

In [35]:
opti.debug.value(X)

array([ 0.392  ,  1.4    ,  2.84   ,  3.84   ,  2.9    ,  1.422  ,
        1.18   ,  1.32392, -1.3808 ,  1.4    ,  0.66   ])

In [34]:
opti.debug.value( (x5*(-x0*((1/2)*omega_max*t_total + phi_0)**2 - x3)))

-3504.886333494073

In [42]:
# log[-1]
log[-1] = log[-1].tolist()
print(log[-1])


[0.00022327413749841384, 19.998702812382657, 0.07202306518860084, 9.475962956753776, 4.586768824001846, 0.038238479633328167, 0.15212563733934145, 0.1879659304814252, -1.3374686493078032, 0.03364603865502786, 0.11388716024090186]


In [ ]:
[0.000128, 20, 0.0516, 9.46, 4.6, 0.0263, 0.015, 0.01906, -1.327, 0.0256, 0.0244]